In [2]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

sess = sagemaker.Session()
bucket = 'sagemaker-multi-label-test'
prefix = 'ic-multilabel'

print('using bucket %s'%bucket)


arn:aws:iam::466363126778:role/AWSGlueServiceSageMakerNotebookRole
using bucket sagemaker-multi-label-test
CPU times: user 123 ms, sys: 8.32 ms, total: 131 ms
Wall time: 172 ms


In [4]:
%%bash
shuf image.lst > im.lst
head -n 647 im.lst > train.lst
tail -n +648 im.lst > val.lst
head train.lst
wc -l train.lst
wc -l val.lst

29046	0	1	asset-image/user-2/node-29046/lake-district-wood-flower-asset-shape-cutout-stamp-embellishment-veneer-blank-plaque-brown.png
176868	1	1	graphic-image/user-111918/node-176868/rose-cluster-graphic-flower-vine-leaf-element-green-pink.png
55138	1	0	asset-image/user-2051/node-55138/autumn-art-watercolour-leaf-3-asset-embellishment-leaves-nature-fall-watercolor-painting-orange.png
188126	0	1	graphic-image/user-2725/node-188126/fresh-pink-flower-01-graphic-element-embellishment-bloom-yellow-white.png
120661	0	1	graphic-image/user-2725/node-120661/flower-template-011-graphic-embellishment-element-floral-bloom-white-gray-black.png
233750	0	1	graphic-image/user-66801/node-233750/reach-for-the-sun-flower-graphic-element-embellishment-floral-silk-peach-yellow-pink.png
253946	1	1	graphic-image/user-111918/node-253946/bright-cheerful-vine-02-graphic-ivy-leaf-flower-bud-element-green-purple.png
114175	0	1	graphic-image/user-2725/node-114175/a-mother-s-love-flower-sticker-4-graphic-flowers-f

In [5]:
s3train_lst = 's3://{}/{}/train_lst/'.format(bucket, prefix)
s3validation_lst = 's3://{}/{}/validation_lst/'.format(bucket, prefix)

!aws s3 cp train.lst $s3train_lst --quiet
!aws s3 cp val.lst $s3validation_lst --quiet

In [12]:
%%bash
mkdir images

In [13]:
import os
import boto3
import botocore




lst = open('image.lst')
s3 = boto3.resource('s3')

images = []


for line in lst: 
    line = line.split()
    s3_image_path = line[len(line) - 1]
    images.append(s3_image_path)

bucket = 'pixelscrapper-user-content'
image_dir = 'images/'

for file in images:
   try:
        s3.Bucket(bucket).download_file(file, image_dir + os.path.basename(file))
   except botocore.exceptions.ClientError as e:
       if e.response['Error']['Code'] == "404":
           print("The object does not exist.")
       else:
           raise


In [27]:
import os
import boto3
import botocore




lst = open('image.lst')
s3 = boto3.resource('s3')
out_lst = open('imageLST.lst', 'w')

for line in lst: 
    line = line.split()
    s3_image_path = line[len(line) - 1].split('/')
    image_file_name = s3_image_path[len(s3_image_path)-1]
    line = line[:3]
    line.append(image_file_name)
    out_lst.write('\t'.join(map(str,line)) + '\n')
        

In [28]:
%%bash
shuf imageLST.lst > im.lst
head -n 647 im.lst > train.lst
tail -n +648 im.lst > val.lst
head train.lst
wc -l train.lst
wc -l val.lst

27235	1	0	leaf-006-template-embellishment-fall-autumn.png
141705	0	0	here-now-alpha-card-yellow-graphic-element-journal-numbers-white-black.png
96160	1	0	falling-for-you-yellow-leaf-heart-4-graphic-element-embellishment-fall-nature.png
155665	1	0	leaves-no-4-leaf-5-graphic-element-embellishment-greenery-green-light-white.png
176326	1	1	mod-flower-02-graphic-dots-leaf-element-orange-yellow-green-olive.png
53387	1	0	autumn-pieces-leaf-05-asset-embellishment-leaves-fall-season-dried-orange.png
116408	1	0	leaf-shape-mask-template-002-graphic-element-embellishment-nature-outdoors-black.png
41471	1	0	one-stop-bunting-shop-leaf-asset-embellishment-silk-extracted-fall-autumn-spring-nature-small-green.png
95618	0	0	new-years-resolutions-i-will-graphic-word-strip-art-element-embellishment-year-s-bucket-list-to-do.png
50959	1	0	summer-daydreams-craft-leaf-02-asset-embellishment.png
647 train.lst
242 val.lst


In [32]:
bucket = 'sagemaker-multi-label-test'
prefix = 'ic-multilabel'

s3train_lst = 's3://{}/{}/train_lst/'.format(bucket, prefix)
s3validation_lst = 's3://{}/{}/validation_lst/'.format(bucket, prefix)

!aws s3 cp train.lst $s3train_lst --quiet
!aws s3 cp val.lst $s3validation_lst --quiet

In [33]:
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)

!aws s3 cp images $s3train --recursive
!aws s3 cp images $s3validation --recursive

upload: images/a-mother-s-love-flower-sticker-4-graphic-flowers-floral-paper-bloom-embellishment-element-red-green.png to s3://sagemaker-multi-label-test/ic-multilabel/train/a-mother-s-love-flower-sticker-4-graphic-flowers-floral-paper-bloom-embellishment-element-red-green.png
upload: images/a-night-in-october-leaf-03a-graphic-element-green.png to s3://sagemaker-multi-label-test/ic-multilabel/train/a-night-in-october-leaf-03a-graphic-element-green.png
upload: images/a-night-in-october-leaf-04b-graphic-element-trail-orange.png to s3://sagemaker-multi-label-test/ic-multilabel/train/a-night-in-october-leaf-04b-graphic-element-trail-orange.png
upload: images/all-the-princesses-elements-leaves-2-graphic-element-embellishment-nature-leaf-green.png to s3://sagemaker-multi-label-test/ic-multilabel/train/all-the-princesses-elements-leaves-2-graphic-element-embellishment-nature-leaf-green.png
upload: images/all-the-princesses-yellow-button-2-graphic-element-embellishment-vintage-round-ornate-sew

upload: images/autumn-art-leaf-04-graphic-fall-element-purple.png to s3://sagemaker-multi-label-test/ic-multilabel/train/autumn-art-leaf-04-graphic-fall-element-purple.png
upload: images/autumn-art-flower-asset-embellishment-2014-blogtrain-fall-season-everyday-life-fabric-orange-gold.png to s3://sagemaker-multi-label-test/ic-multilabel/train/autumn-art-flower-asset-embellishment-2014-blogtrain-fall-season-everyday-life-fabric-orange-gold.png
upload: images/autumn-day-green-flower-graphic-floral-element-fabric-olive.png to s3://sagemaker-multi-label-test/ic-multilabel/train/autumn-day-green-flower-graphic-floral-element-fabric-olive.png
upload: images/autumn-bramble-enamel-leaf-graphic-heart-element-embellishment-tan.png to s3://sagemaker-multi-label-test/ic-multilabel/train/autumn-bramble-enamel-leaf-graphic-heart-element-embellishment-tan.png
upload: images/autumn-art-october-mini-painted-leaf-green-violet-graphic-element-purple.png to s3://sagemaker-multi-label-test/ic-multilabel/tra

upload: images/bedouin-flower-green-asset-embellishment-bedouin-nights-kit.png to s3://sagemaker-multi-label-test/ic-multilabel/train/bedouin-flower-green-asset-embellishment-bedouin-nights-kit.png
upload: images/bearly-spring-leaf-01-graphic-greenery-element-green.png to s3://sagemaker-multi-label-test/ic-multilabel/train/bearly-spring-leaf-01-graphic-greenery-element-green.png
upload: images/bingo-chip-extras-period-graphic-element-embellishment-wood-button-vintage-antique-marker-game-toy.png to s3://sagemaker-multi-label-test/ic-multilabel/train/bingo-chip-extras-period-graphic-element-embellishment-wood-button-vintage-antique-marker-game-toy.png
upload: images/bingo-chip-extras-leaf-01-graphic-element-embellishment-wood-button-vintage-antique-marker-game-toy.png to s3://sagemaker-multi-label-test/ic-multilabel/train/bingo-chip-extras-leaf-01-graphic-element-embellishment-wood-button-vintage-antique-marker-game-toy.png
upload: images/bird-206-graphic-element-template-illustration-st

upload: images/brown-paper-flower-3-asset-embellishment-flowers-sparkle-tan.png to s3://sagemaker-multi-label-test/ic-multilabel/train/brown-paper-flower-3-asset-embellishment-flowers-sparkle-tan.png
upload: images/cambodia-flower-set-4e-asset-embellishment-glitter-white-green-blue.png to s3://sagemaker-multi-label-test/ic-multilabel/train/cambodia-flower-set-4e-asset-embellishment-glitter-white-green-blue.png
upload: images/cambodia-light-blue-glitter-leaf-asset-flower-embellishment.png to s3://sagemaker-multi-label-test/ic-multilabel/train/cambodia-light-blue-glitter-leaf-asset-flower-embellishment.png
upload: images/cambodia-flower-set-5d-asset-embellishment-glitter-white-gold.png to s3://sagemaker-multi-label-test/ic-multilabel/train/cambodia-flower-set-5d-asset-embellishment-glitter-white-gold.png
upload: images/cast-a-spell-elements-number-card-asset-other-embellishment-autumn-fall-season-halloween-text-1-red.png to s3://sagemaker-multi-label-test/ic-multilabel/train/cast-a-spell

upload: images/christmas-in-july-berry-branch-wood-asset-leaf-embellishment-berries-holly-red-white.png to s3://sagemaker-multi-label-test/ic-multilabel/train/christmas-in-july-berry-branch-wood-asset-leaf-embellishment-berries-holly-red-white.png
upload: images/confidence-word-snippet-begin-graphic-art-element-white-black.jpg to s3://sagemaker-multi-label-test/ic-multilabel/train/confidence-word-snippet-begin-graphic-art-element-white-black.jpg
upload: images/clover-accordion-flower-template-004-embellishment-st-patricks-day-lucky-luck-irish-ireland-shamrock.png to s3://sagemaker-multi-label-test/ic-multilabel/train/clover-accordion-flower-template-004-embellishment-st-patricks-day-lucky-luck-irish-ireland-shamrock.png
upload: images/colorabstract-leaf-1-graphic-leaf-silk-element-green.png to s3://sagemaker-multi-label-test/ic-multilabel/train/colorabstract-leaf-1-graphic-leaf-silk-element-green.png
upload: images/chinese-new-year-bamboo-asset-leaf-embellishment-china-asian-decoration

...

upload: images/scotland-stamp-001o-graphic-element-template-ephemera-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/scotland-stamp-001o-graphic-element-template-ephemera-black.png
upload: images/scraps-3-enamel-dot-4-graphic-element-flair-orange-yellow-pink.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/scraps-3-enamel-dot-4-graphic-element-flair-orange-yellow-pink.png
upload: images/sas-scrapkit-happynewyear-element08-graphic-element-paper-flower-gray.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/sas-scrapkit-happynewyear-element08-graphic-element-paper-flower-gray.png
upload: images/scotland-stamp-001n-graphic-element-template-ephemera-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/scotland-stamp-001n-graphic-element-template-ephemera-black.png
upload: images/school-fun-flower-2-graphic-education-teacher-element-red-orange-green-teal-white-brown-tan.png to s3://sagemaker-multi-label-test/ic-multilabel/va

upload: images/softly-falling-felt-doodle-5-graphic-element-leaf-cream.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/softly-falling-felt-doodle-5-graphic-element-leaf-cream.png
upload: images/spookalicious-element-templates-glitter-leafy-branch-template-leaf-embellishment.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/spookalicious-element-templates-glitter-leafy-branch-template-leaf-embellishment.png
upload: images/special-day-leaf-2-graphic-element-green.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/special-day-leaf-2-graphic-element-green.png
upload: images/stitch-doodle-template-019-graphic-element-embellishment-sew-sewing-thread-stem-flower-plant-leaf.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/stitch-doodle-template-019-graphic-element-embellishment-sew-sewing-thread-stem-flower-plant-leaf.png
upload: images/spring-day-templates-leaf-graphic-element-embellishment-template-nature-gray.png to s3://sagemaker-mu

upload: images/summer-daydreams-pink-bud-asset-flower-embellishment-white-greenish.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/summer-daydreams-pink-bud-asset-flower-embellishment-white-greenish.png
upload: images/summer-day-doodle-13-template-graphic-printable-hybrid-textured-element-stamp-flower-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/summer-day-doodle-13-template-graphic-printable-hybrid-textured-element-stamp-flower-black.png
upload: images/stitched-leaf-template-2-stitches-embellishment-autumn-fall-leaves-sew-sewing-outline.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/stitched-leaf-template-2-stitches-embellishment-autumn-fall-leaves-sew-sewing-outline.png
upload: images/summer-day-sticker-doodle-6d-graphic-element-flower-blue-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/summer-day-sticker-doodle-6d-graphic-element-flower-blue-black.png
upload: images/strawberry-fields-leaf-doodle-charm-0

upload: images/the-good-life-december-stamps-swirls-1-stamp-graphic-element-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/the-good-life-december-stamps-swirls-1-stamp-graphic-element-black.png
upload: images/the-good-life-april-2020-travel-stamps-passport-template-graphic-element-world-word-art-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/the-good-life-april-2020-travel-stamps-passport-template-graphic-element-world-word-art-black.png
upload: images/the-best-is-yet-to-come-2017-leaf-2-graphic-felt-stitching-scrumptiously-element-pocket-scrapping.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/the-best-is-yet-to-come-2017-leaf-2-graphic-felt-stitching-scrumptiously-element-pocket-scrapping.png
upload: images/the-good-life-december-2020-christmas-elements-flower-02-graphic-button-element-lime-green-gray.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/the-good-life-december-2020-christmas-elements-flower-02-g

upload: images/watercolor-8-flower-13-color-graphic-element-blended-orange-peach.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/watercolor-8-flower-13-color-graphic-element-blended-orange-peach.png
upload: images/winter-arabesque-floral-heart-graphic-snow-snowflakes-ballet-dance-january-2015-blog-train-flower.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/winter-arabesque-floral-heart-graphic-snow-snowflakes-ballet-dance-january-2015-blog-train-flower.png
upload: images/watercolor-brush-5-30-graphic-flower-paint-element-template-black.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/watercolor-brush-5-30-graphic-flower-paint-element-template-black.png
upload: images/vintage-memories-genealogy-cluster-3-no-shadow-graphic-element-embellishment-leaf-flower-floral.png to s3://sagemaker-multi-label-test/ic-multilabel/validation/vintage-memories-genealogy-cluster-3-no-shadow-graphic-element-embellishment-leaf-flower-floral.png
upload: images/w

In [50]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print (training_image)


The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:1


In [51]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
multilabel_ic = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         instance_count=1, 
                                         instance_type='ml.p2.xlarge',
                                         volume_size = 50,
                                         max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [52]:


multilabel_ic.set_hyperparameters(num_layers=18,
                             use_pretrained_model=1,
                             image_shape = "3,224,224",
                             num_classes=2,
                             mini_batch_size=20,
                             resize=256,
                             epochs=5,
                             learning_rate=0.001,
                             num_training_samples=647,
                             use_weighted_loss=1,
                             augmentation_type = 'crop_color_transform',
                             precision_dtype='float32',
                             multi_label=1)



In [53]:
train_data = sagemaker.inputs.TrainingInput(s3train, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data = sagemaker.inputs.TrainingInput(s3validation, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
train_data_lst = sagemaker.inputs.TrainingInput(s3train_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data_lst = sagemaker.inputs.TrainingInput(s3validation_lst, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data, 'train_lst': train_data_lst, 
                 'validation_lst': validation_data_lst}



In [54]:
multilabel_ic.fit(inputs=data_channels, logs=True)

2021-02-25 08:13:56 Starting - Starting the training job...
2021-02-25 08:14:20 Starting - Launching requested ML instancesProfilerReport-1614240836: InProgress
......
2021-02-25 08:15:22 Starting - Preparing the instances for training............
2021-02-25 08:17:23 Downloading - Downloading input data.........
2021-02-25 08:18:57 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[02/25/2021 08:19:02 INFO 139987205756032] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[02/25/2021 08:19:02 INFO 139987205756032] Merging with provi

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
[02/25/2021 08:19:37 INFO 139987205756032] Done creating record files...
[02/25/2021 08:19:37 INFO 139987205756032] label-format is multi-hot
[02/25/2021 08:19:37 INFO 139987205756032] use_pretrained_model: 1
[02/25/2021 08:19:37 INFO 139987205756032] multi_label: 1
[02/25/2021 08:19:37 INFO 139987205756032] Using pretrained model for initializing weights and transfer learning.
[02/25/2021 08:19:37 INFO 139987205756032] ---- Parameters ----
[02/25/2021 08:19:37 INFO 139987205756032] num_layers: 18
[02/25/2021 08:19:37 INFO 139987205756032] data type: <type 'numpy.float32'>
[02/25/2021 

In [4]:


ic_classifier = multilabel_ic.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.m4.xlarge')

NameError: name 'multilabel_ic' is not defined

In [5]:



import boto3

import json
 
endpoint = 'image-classification-2021-02-25-08-23-10-956'
 
runtime = boto3.Session().client('sagemaker-runtime')
 
image = 'images/at-the-farm-flower-red-asset-embellishment-country-silk.png'
# Read image into memory
with open(image, 'rb') as f:
    payload = f.read()
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)

# Unpack response
result = json.loads(response['Body'].read().decode())

In [6]:
print(result)

[0.8064126968383789, 0.7896784543991089]


In [7]:
image = 'images/baby-on-board-elements-paper-flower-graphic-element-embellishment-rolled-flowers-pink.png'
# Read image into memory
with open(image, 'rb') as f:
    payload = f.read()
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)

# Unpack response
result = json.loads(response['Body'].read().decode())
print(result)

[0.11923378705978394, 0.8508678078651428]


In [8]:
image = 'images/bearly-spring-leaf-01-graphic-greenery-element-green.png'
# Read image into memory
with open(image, 'rb') as f:
    payload = f.read()
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)

# Unpack response
result = json.loads(response['Body'].read().decode())
print(result)

[0.9451475739479065, 0.25192198157310486]


In [9]:
image = 'images/changse-vellum-feather-graphic-cutout-element-embellishment-white.png'
# Read image into memory
with open(image, 'rb') as f:
    payload = f.read()
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)

# Unpack response
result = json.loads(response['Body'].read().decode())
print(result)

[0.625129222869873, 0.6988133788108826]
